In [2]:
import torch
import pandas
import numpy as np

In [3]:
import pandas
#dataset taken from https://www.kaggle.com/yashsawarn/wifi-stretgth-for-rooms

def read_dataset(csv_name = 'wifi_localization.txt'):
    """
    Reads a csv dataset
    returns it as a pytorch tensor
    """
    data_frame = pandas.read_table(csv_name, delim_whitespace=True, names=('A', 'B', 'C', 'D','E', 'F', 'G', 'ROOM'),
                       dtype={'A': np.int64, 'B': np.float64, 'C': np.float64, 'D': np.float64,'E': np.float64,'F': np.float64,'G': np.float64,'ROOM': np.float64})

    targets_torch = torch.tensor(data_frame['ROOM'].values)
    dataset_torch = torch.tensor(data_frame.values)

    return dataset_torch
dataset_torch = read_dataset()



/tmp/ipython-input-2369501803.py:9: FutureWarning: The 'delim_whitespace' keyword in pd.read_table is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data_frame = pandas.read_table(csv_name, delim_whitespace=True, names=('A', 'B', 'C', 'D','E', 'F', 'G', 'ROOM'),


FileNotFoundError: [Errno 2] No such file or directory: 'wifi_localization.txt'

In [4]:
print(dataset_torch)

NameError: name 'dataset_torch' is not defined

In [15]:
dataset_example = torch.tensor([[3, 22.0, 7.2, 1], [1, 38, 71.3, 1], [3, 26, 7.9, 1], [1, 35, 53.1, 1]])
print("dataset_example \n", dataset_example)


dataset_example 
 tensor([[ 3.0000, 22.0000,  7.2000,  1.0000],
        [ 1.0000, 38.0000, 71.3000,  1.0000],
        [ 3.0000, 26.0000,  7.9000,  1.0000],
        [ 1.0000, 35.0000, 53.1000,  1.0000]])


In [6]:
root_node = Node_CART()
root_node.data_torch_partition = dataset_example

print("root node \n ", root_node.data_torch_partition)
xml_root_node = root_node.to_xml()

print("xml_root_node \n", xml_root_node)

NameError: name 'Node_CART' is not defined

In [9]:
root_node.threshold_value = 3
root_node.feature_num = 0

#indices of left and right partitions
left_idxs = dataset_example[:, root_node.feature_num] < root_node.threshold_value
right_idxs = dataset_example[:, root_node.feature_num] >= root_node.threshold_value
#data partitions
dataset_partition_left = dataset_example[left_idxs]
dataset_partition_right = dataset_example[right_idxs]

print("dataset_partition_left \n", dataset_partition_left)
print("dataset_partition_right \n", dataset_partition_right)
#create left child
left_child = Node_CART(current_depth = 1)
left_child.data_torch_partition = dataset_partition_left
root_node.node_left = left_child
#create right child
right_child = Node_CART(current_depth = 1)
right_child.data_torch_partition = dataset_partition_right
root_node.node_right = right_child
#write xml example
root_node.ref_CART = root_node
xml_string = root_node.to_xml()

#print(xml_string)
file = open("example1.xml", "a")
file.write(xml_string)
file.close()


NameError: name 'root_node' is not defined

In [20]:
print(xml_string)

<node><thresh>3</thresh><feature>0</feature><depth>0</depth><gini>0</gini><node><thresh>0</thresh><feature>0</feature><depth>1</depth><gini>0</gini><dominant_class>None</dominant_class><acc_dominant_class>None</acc_dominant_class></node><node><thresh>0</thresh><feature>0</feature><depth>1</depth><gini>0</gini><dominant_class>None</dominant_class><acc_dominant_class>None</acc_dominant_class></node></node>


In [111]:

class Node_CART:
    def __init__(self, num_classes=4, ref_CART=None, current_depth=0):
        self.ref_CART = ref_CART
        self.threshold_value = 0
        self.feature_num = 0
        self.node_right = None
        self.node_left = None
        self.data_torch_partition = None
        self.gini = 0
        self.dominant_class = None
        self.accuracy_dominant_class = None
        self.num_classes = num_classes
        self.current_depth = current_depth
        self.max_depth = None  # Optional: set externally if needed

    def to_xml(self, current_str=""):
        str_node = "<node><thresh>" + str(self.threshold_value) + "</thresh>" + "<feature>" + str(self.feature_num) + "</feature><depth>" + str(self.current_depth)+ "</depth>"
        str_node += "<gini>" + str(self.gini) + "</gini>"
        if self.node_right is not None:
            str_node += self.node_right.to_xml(current_str)
        if self.node_left is not None:
            str_node += self.node_left.to_xml(current_str)
        if self.is_leaf():
            str_node += "<dominant_class>" + str(self.dominant_class) + "</dominant_class><acc_dominant_class>"  + str(self.accuracy_dominant_class) + "</acc_dominant_class>"
        str_node += "</node>"
        return str_node

    def is_leaf(self):
        return (self.node_left is None and self.node_right is None)

    @staticmethod
    def calculate_gini(data_partition_torch, num_classes=2):
        if data_partition_torch.numel() == 0:
            return 0.0
        labels = data_partition_torch[:, -1].long()
        # Cuenta cuántos ejemplos hay de cada clase
        counts = torch.bincount(labels, minlength=num_classes)
        probs = counts.float() / counts.sum()
        gini = 1.0 - torch.sum(probs ** 2)
        return gini.item()

    def create_with_children(self, data_torch, current_depth, min_gini=0.000001):
        labels = data_torch[:, -1].long()
        counts = torch.bincount(labels, minlength=self.num_classes).float()
        self.dominant_class = torch.argmax(counts).item()

        # If pure node, make leaf
        if (counts > 0).sum() == 1:
            print('Pure node, exit')
            return self

        # If max depth reached
        if self.max_depth is not None and current_depth >= self.max_depth:
            print("Reached Max_depth")
            return self

        # Select best split
        feature, thresh, gini = self.select_best_feature_and_thresh(data_torch, self.num_classes)
        # If no good split, make leaf
        if feature is None or gini < min_gini:
            print('No good split, exit')
            return self

        self.feature_num = feature
        self.threshold_value = thresh
        self.gini = gini

        # Partition data
        left_mask = data_torch[:, feature] < thresh
        right_mask = ~left_mask

        left_partition = data_torch[left_mask]
        right_partition = data_torch[right_mask]

        # Create child nodes
        self.node_left = Node_CART(num_classes=self.num_classes, current_depth=current_depth + 1)
        self.node_right = Node_CART(num_classes=self.num_classes, current_depth=current_depth + 1)
        self.node_left.max_depth = self.max_depth
        self.node_right.max_depth = self.max_depth

        self.node_left.create_with_children(left_partition, current_depth + 1, min_gini)
        self.node_right.create_with_children(right_partition, current_depth + 1, min_gini)

        return self

    def select_best_feature_and_thresh(self, data_torch, num_classes=2):
        n_samples, n_features = data_torch.shape[0], data_torch.shape[1] - 1
        best_gini = float('inf')
        best_feature = None
        best_thresh = None

        for feature_idx in range(n_features):
            feature_values = data_torch[:, feature_idx]
            sorted_vals = torch.sort(feature_values.unique())[0]
            # Only try thresholds between unique values
            if len(sorted_vals) > 1:
                thresholds = (sorted_vals[:-1] + sorted_vals[1:]) / 2
            else:
                thresholds = sorted_vals  # Only one value, no split possible

            for thresh in thresholds:
                left_mask = feature_values < thresh
                right_mask = ~left_mask

                left_partition = data_torch[left_mask]
                right_partition = data_torch[right_mask]

                gini_left = self.calculate_gini(left_partition, num_classes)
                gini_right = self.calculate_gini(right_partition, num_classes)

                weighted_gini = (left_partition.size(0) * gini_left +
                                 right_partition.size(0) * gini_right) / n_samples

                if weighted_gini < best_gini:
                    best_gini = weighted_gini
                    best_feature = feature_idx
                    best_thresh = thresh.item()

        return best_feature, best_thresh, best_gini

    def evaluate_node(self, input_torch):
        feature_val_input = input_torch[self.feature_num]
        if self.is_leaf():
            return self.dominant_class
        else:
            if feature_val_input < self.threshold_value:
                return self.node_left.evaluate_node(input_torch)
            else:
                return self.node_right.evaluate_node(input_torch)

def test_CART(root_node, D):
    """
    Evalúa un árbol CART previamente entrenado (root_node) sobre un conjunto de datos D (tensor).
    Calcula y retorna la tasa de aciertos (accuracy), definida como:
        accuracy = c / n
    donde:
        c = número de estimaciones correctas (predicción == etiqueta real)
        n = número total de muestras

    Parámetros:
        root_node (Node_CART): nodo raíz del árbol entrenado.
        D (torch.Tensor): conjunto de datos, última columna es la etiqueta.

    Retorna:
        float: tasa de aciertos (accuracy).
    """
    # Contador de aciertos
    correct = 0
    n = D.shape[0]

    # Para cada muestra en D
    for i in range(n):
        sample = D[i, :-1]  # Todas las columnas menos la última (features)
        true_label = D[i, -1].item()  # Última columna (etiqueta real)
        pred_label = root_node.evaluate_node(sample)  # Predicción del árbol

        if pred_label == true_label:
            correct += 1

    accuracy = correct / n
    return accuracy

In [95]:
def calculate_gini(node_left, node_right):
  size_left = node_left.data_torch_partition.shape[0]
  size_right = node_right.data_torch_partition.shape[0]
  size_total = size_left + size_right
  gini_left = 1 - (((node_left.data_torch_partition[:, 3] == 0).sum().item() / size_left)**2 + ((node_left.data_torch_partition[:, 3] == 1).sum().item()/size_left)**2)
  gini_right = 1 - (((node_right.data_torch_partition[:, 3] == 0).sum().item() / size_right)**2 + ((node_right.data_torch_partition[:, 3] == 1).sum().item()/size_right)**2)
  gini_total = (size_left / size_total * gini_left) + (size_right / size_total * gini_right)
  print("gini_left: ", gini_left, "gini_right: ", gini_right, "gini_total: ", gini_total)
  return gini_total
gini_total = calculate_gini(left_child, right_child)
print("gini_total ", gini_total)

NameError: name 'left_child' is not defined

#Unit Tests función: calculate_gini

In [136]:
# Prueba unitaria 1: Gini para un solo grupo homogéneo (impureza debe ser 0)
def test_gini_homogeneous():
    node = Node_CART(num_classes=2)
    # Todos los elementos son de la clase 0
    data = torch.tensor([[1, 2, 0], [2, 3, 0], [3, 4, 0]], dtype=torch.float32)
    gini = node.calculate_gini(data, num_classes=2)
    print('Test 1 - Gini homogéneo:', gini)
    assert abs(gini - 0.0) < 1e-6, f"Esperado 0.0, obtenido {gini}"

# Prueba unitaria 2: Gini para dos clases balanceadas (impureza máxima)
def test_gini_balanced():
    node = Node_CART(num_classes=2)
    # Mitad clase 0, mitad clase 1
    data = torch.tensor([[1, 2, 0], [2, 3, 1], [3, 4, 0], [4, 5, 1]], dtype=torch.float32)
    gini = node.calculate_gini(data, num_classes=2)
    print('Test 2 - Gini balanceado:', gini)
    assert abs(gini - 0.5) < 1e-6, f"Esperado 0.5, obtenido {gini}"

# Ejecutar pruebas
test_gini_homogeneous()
test_gini_balanced()

Test 1 - Gini homogéneo: 0.0
Test 2 - Gini balanceado: 0.5


# Unit Tests función: select_best_feature_and_thresh

In [139]:
import torch

def test_select_best_feature_and_thresh():
    # El feature 0 separa perfectamente las clases con threshold 2
    # [feature0, feature1, clase]
    data = torch.tensor([
        [1.0, 10.0, 0],
        [2.0, 20.0, 0],
        [3.0, 30.0, 1],
        [4.0, 40.0, 1]
    ])

    node = Node_CART(num_classes=2)
    best_feature, best_thresh, best_gini = node.select_best_feature_and_thresh(data, num_classes=2)

    print("Mejor feature:", best_feature)
    print("Mejor threshold:", best_thresh)
    print("Mejor gini:", best_gini)


    # Esperamos que el mejor feature sea 0 y el mejor threshold sea 2.0 entre 3.0 (ambos separan perfectamente)
    assert best_feature == 0, "El mejor feature debería ser la columna 0"
    assert best_thresh < 3, "El mejor threshold debería ser entre 2.0 y 3.0"
    assert best_gini == 0.0, "El gini debería ser 0 para una separación perfecta"



def test_select_best_feature_and_thresh_feature2():
    # Dataset: solo la columna 2 permite separación perfecta con threshold 12.5
    # [feature0, feature1, feature2, clase]
    data = torch.tensor([
        [0.0, 1.0, 10.0, 1],
        [0.0, 1.0, 11.0, 1],
        [0.0, 1.0, 12.0, 1],
        [0.0, 1.0, 13.0, 0],
        [0.0, 1.0, 14.0, 0],
        [0.0, 1.0, 15.0, 0],
        [0.0, 1.0, 16.0, 0],
        [0.0, 1.0, 17.0, 0],
    ])

    node = Node_CART(num_classes=2)
    best_feature, best_thresh, best_gini = node.select_best_feature_and_thresh(data, num_classes=2)

    print("Mejor feature:", best_feature)
    print("Mejor threshold:", best_thresh)
    print("Mejor gini:", best_gini)

    # Esperamos que el mejor feature sea 2 (columna 2) y threshold  13.0
    assert best_feature == 2, "El mejor feature debería ser la columna 2"

    assert best_thresh < 13.0 , "El mejor threshold debería ser entre 12 y 13"
    assert best_gini == 0.0, "El gini debería ser 0 para una separación perfecta"

print(" *** Unit test de select_best_feature_and_thresh ")
print("Test #1 de la función select_best_feature_and_thresh. El feature 0 separa perfectamente las clases con threshold 2.5")
test_select_best_feature_and_thresh()

print("_____________________________________________________")

print("Test #1 de la función select_best_feature_and_thresh. El feature 0 separa perfectamente las clases con threshold 2.5")
test_select_best_feature_and_thresh_feature2()

 *** Unit test de select_best_feature_and_thresh 
Test #1 de la función select_best_feature_and_thresh. El feature 0 separa perfectamente las clases con threshold 2.5
Mejor feature: 0
Mejor threshold: 2.5
Mejor gini: 0.0
_____________________________________________________
Test #1 de la función select_best_feature_and_thresh. El feature 0 separa perfectamente las clases con threshold 2.5
Mejor feature: 2
Mejor threshold: 12.5
Mejor gini: 0.0


#Unit Test función: test_card

In [134]:
def test_CART_trivial():
    # Árbol hoja que siempre predice 1
    class DummyNode:
        def evaluate_node(self, x):
            return 1

    D = torch.tensor([
        [1.0, 2.0, 1],
        [2.0, 3.0, 0],
        [3.0, 4.0, 1],
        [4.0, 5.0, 1],
    ])
    dummy_root = DummyNode()
    acc = test_CART(dummy_root, D)
    print("Accuracy árbol trivial:", acc)
    assert acc == 0.75, "El accuracy debería ser 0.75"

def test_CART_simple_CART():
    # Dataset perfectamente separable por feature 0 con threshold 2.0
    data = torch.tensor([
        [1.0, 10.0, 0],
        [2.0, 20.0, 0],
        [3.0, 30.0, 1],
        [4.0, 40.0, 1]
    ])

    node = Node_CART(num_classes=2)
    node.create_with_children(data, current_depth=0, min_gini=0.0)
    acc = test_CART(node, data)  # accuracy
    print("Accuracy árbol perfectamente valanceado:", acc)
    assert acc == 1.0, "El accuracy debería ser 1"

def test_CART_DEPTH_2():
  data = torch.tensor([
      [1.0, 0.0, 1.0, 0],
      [1.0, 0.0, 2.0, 0],
      [1.0, 0.0, 3.0, 1],
      [1.0, 0.0, 4.0, 0],
      [1.0, 0.0, 5.0, 0],
      [1.0, 0.0, 6.0, 0],
  ])

  node = Node_CART(num_classes=2)
  node.create_with_children(data, current_depth=0)
  acc = test_CART(node, data)  # accuracy
  print("Accuracy árbol usando un min gini mayor a 0", acc)
  assert acc < 1.0 and acc >0, "El accuracy debería ser menor que uno y mayor a 0, usando un min gini mayor a 0"

def test_CART_DEPTH_2_Gin0():
  data = torch.tensor([
      [1.0, 0.0, 1.0, 0],
      [1.0, 0.0, 2.0, 0],
      [1.0, 0.0, 3.0, 1],
      [1.0, 0.0, 4.0, 0],
      [1.0, 0.0, 5.0, 0],
      [1.0, 0.0, 6.0, 0],
  ])

  node = Node_CART(num_classes=2)
  node.create_with_children(data, current_depth=0, min_gini=0)
  acc = test_CART(node, data)  # accuracy
  print("Accuracy árbol usando un min gini 0", acc)
  assert acc == 1 , "El accuracy debería ser igual a 1"

test_CART_simple_CART()
test_CART_DEPTH_2()
test_CART_DEPTH_2_Gin0()

Select the best split  2
feature  0
thresh  2.5
gini  0.0
Pure node, exit
Pure node, exit
Accuracy árbol perfectamente valanceado: 1.0
Select the best split  2
feature  2
thresh  3.5
gini  0.2222222089767456
Select the best split  2
feature  2
thresh  2.5
gini  0.0
No good split, exit
Pure node, exit
Accuracy árbol usando un min gini mayor a 0 0.8333333333333334
Select the best split  2
feature  2
thresh  3.5
gini  0.2222222089767456
Select the best split  2
feature  2
thresh  2.5
gini  0.0
Pure node, exit
Pure node, exit
Pure node, exit
Accuracy árbol usando un min gini 0 1.0


In [128]:
dataset_example = torch.tensor([
    [1.0, 0.0, 1.0, 0],
    [1.0, 0.0, 2.0, 0],
    [1.0, 0.0, 3.0, 1],
    [1.0, 0.0, 4.0, 0],
    [1.0, 0.0, 5.0, 0],
    [1.0, 0.0, 6.0, 0],
])
newNode = Node_CART(num_classes=2)
newNode.create_with_children(dataset_example, 0)
xml_string =newNode.to_xml()

#print(xml_string)
#file = open("example1.xml", "a")
#file.write(xml_string)
#file.close()
print(xml_string)

Select the best split  2
feature  2
thresh  3.5
gini  0.2222222089767456
Select the best split  2
feature  2
thresh  2.5
gini  0.0
No good split, exit
Pure node, exit
<node><thresh>3.5</thresh><feature>2</feature><depth>0</depth><gini>0.2222222089767456</gini><node><thresh>0</thresh><feature>0</feature><depth>1</depth><gini>0</gini><dominant_class>0</dominant_class><acc_dominant_class>None</acc_dominant_class></node><node><thresh>0</thresh><feature>0</feature><depth>1</depth><gini>0</gini><dominant_class>0</dominant_class><acc_dominant_class>None</acc_dominant_class></node></node>
